# Hacker News Posts Analysis

We will compare two types of posts from the [Hacker News](https://news.ycombinator.com) web forum.  Hacker News contains posts and comments on anything technology related.  This project will analyze two types of posts:  "Ask HN" and "Show HN."

"Ask HN" posts questions to ask the Hacker News community.  "Show HN" shares a project, product, or anything interesting to the Hacker News community.

We will be exploring the following two questions:
* Which of these two types receive more comments on average?
* Do posts created at a certain time receive more comments on average?

To analyze this data, we obtained a [dataset file](https://www.kaggle.com/hacker-news/hacker-news-posts/home/) containing posts submissions on the Hacker News web site.

Note that this data set was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments, and then randomly sampling from the remaining submissions.

## Exploring the Data Set
The following describe the fields in this database:
* id -- Unique identifier for the Hacker News post.
* title -- Title of the post.
* url -- URL the post links to, if the post has a URL.
* num_points -- Number of points the post acquired, calculated as the total number of up votes minus the total number of down votes.
* num_comments -- Number of comments made on the post.
* author -- Username of the author who submitted the post.
* created_at -- Date and time the post was submitted.

First we'll explore a sample of the data and examine the header:

In [1]:
from csv import reader

hn = list(reader(open("hacker_news.csv", encoding="UTF8")))

hn[:6]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12579008',
  'You have two days to comment if you want stem cells to be classified as your own',
  'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018',
  '1',
  '0',
  'altstar',
  '9/26/2016 3:26'],
 ['12579005',
  'SQLAR  the SQLite Archiver',
  'https://www.sqlite.org/sqlar/doc/trunk/README.md',
  '1',
  '0',
  'blacksqr',
  '9/26/2016 3:24'],
 ['12578997',
  'What if we just printed a flatscreen television on the side of our boxes?',
  'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43',
  '1',
  '0',
  'pavel_lishin',
  '9/26/2016 3:19'],
 ['12578989',
  'algorithmic music',
  'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext',
  '1',
  '0',
  'poindontcare',
  '9/26/2016 3:16'],
 ['12578979',
  'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake',
  'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94',
  

## Separating Heading From the Data Set

Let's separate the heading from the rest of the data set.  We'll store the headers in a separate variable.

In [2]:
headers = hn[0]
hn = hn[1:]

print(headers)
hn[:5]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12579008',
  'You have two days to comment if you want stem cells to be classified as your own',
  'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018',
  '1',
  '0',
  'altstar',
  '9/26/2016 3:26'],
 ['12579005',
  'SQLAR  the SQLite Archiver',
  'https://www.sqlite.org/sqlar/doc/trunk/README.md',
  '1',
  '0',
  'blacksqr',
  '9/26/2016 3:24'],
 ['12578997',
  'What if we just printed a flatscreen television on the side of our boxes?',
  'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43',
  '1',
  '0',
  'pavel_lishin',
  '9/26/2016 3:19'],
 ['12578989',
  'algorithmic music',
  'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext',
  '1',
  '0',
  'poindontcare',
  '9/26/2016 3:16'],
 ['12578979',
  'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake',
  'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94',
  '1',
  '0',
  'markgainor1',
  '9/26/2016 3:14']]

## Extracting "Ask HN" and "Show HN" Posts

We are interested in the title of the posts, the number of comments for each post, and the date the post was created.

We'll store the "Ask HN" posts in the ask_posts variable and "Show HN" posts in the show_posts variable.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn") :
        ask_posts.append(row)
    elif title.lower().startswith("show hn") :
        show_posts.append(row)
    else :
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

9139
10158
273822


Looks like there are 9,139 "Ask HN" posts and 10,158 "Show HN" posts.

## Calculating the Average Number of Comments for Ask HN and Show HN Posts
Now that we have separated the Ask and Show HN posts, calculation of the average number of comments can begin:

In [4]:
total_ask_comments = 0

for ask_post in ask_posts:
    total_ask_comments += int(ask_post[4])

avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

10.393478498741656


In [5]:
total_show_comments = 0

for show_post in show_posts:
    total_show_comments += int(show_post[4])

avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

4.886099625910612


## Analysis of Show Posts vs Ask Posts

It seems as though the Ask posts have more comments on average than Show posts.  The Ask posts average about 14 comments per post versus about 5 comments per Show post.  Since the "Ask" posts clearly contains more comments, we'll focus the rest of our analysis on the "Ask HN" posts.

## Finding the Amount of Ask HN Posts and Comments by Hour Created
To answer our second question as stated in the introduction, we'll determine the amount of comments an "Ask HN" post receives by a certain creation time.

First, we'll calculate the number of posts created during each hour of day, along with the number of comments for these posts.
Second, we'll calculate the average number of comments each "Ask HN" posts recieves at each of hour of the day.

In [6]:
import datetime as dt

result_list = []

for ask_post in ask_posts:
    result_list.append([ask_post[6], int(ask_post[4])])

counts_by_hour = {}
comments_by_hour = {}

for result in result_list:
    hr = dt.datetime.strptime(result[0], "%m/%d/%Y %H:%M").strftime("%H")
        
    if hr in counts_by_hour:
        counts_by_hour[hr] += 1
        comments_by_hour[hr] += result[1]
    else:
        counts_by_hour[hr] = 1
        comments_by_hour[hr] = result[1]

comments_by_hour

{'02': 2996,
 '01': 2089,
 '22': 3372,
 '21': 4500,
 '19': 3954,
 '17': 5547,
 '15': 18525,
 '14': 4972,
 '13': 7245,
 '11': 2797,
 '10': 3013,
 '09': 1477,
 '07': 1585,
 '03': 2154,
 '23': 2297,
 '20': 4462,
 '16': 4466,
 '08': 2362,
 '00': 2277,
 '18': 4877,
 '12': 4234,
 '04': 2360,
 '06': 1587,
 '05': 1838}

## Calculate Average Number of Comments for Ask HN Posts per Hour

In [7]:
avg_by_hour = []

for hr in counts_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['02', 11.137546468401487],
 ['01', 7.407801418439717],
 ['22', 8.804177545691905],
 ['21', 8.687258687258687],
 ['19', 7.163043478260869],
 ['17', 9.449744463373083],
 ['15', 28.676470588235293],
 ['14', 9.692007797270955],
 ['13', 16.31756756756757],
 ['11', 8.96474358974359],
 ['10', 10.684397163120567],
 ['09', 6.653153153153153],
 ['07', 7.013274336283186],
 ['03', 7.948339483394834],
 ['23', 6.696793002915452],
 ['20', 8.749019607843136],
 ['16', 7.713298791018998],
 ['08', 9.190661478599221],
 ['00', 7.5647840531561465],
 ['18', 7.94299674267101],
 ['12', 12.380116959064328],
 ['04', 9.7119341563786],
 ['06', 6.782051282051282],
 ['05', 8.794258373205741]]

## Sorting Values from a List of Lists
We'll sort the average number of comments for "Ask HN" posts per hour by the highest average value first.

In [8]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[11.137546468401487, '02'], [7.407801418439717, '01'], [8.804177545691905, '22'], [8.687258687258687, '21'], [7.163043478260869, '19'], [9.449744463373083, '17'], [28.676470588235293, '15'], [9.692007797270955, '14'], [16.31756756756757, '13'], [8.96474358974359, '11'], [10.684397163120567, '10'], [6.653153153153153, '09'], [7.013274336283186, '07'], [7.948339483394834, '03'], [6.696793002915452, '23'], [8.749019607843136, '20'], [7.713298791018998, '16'], [9.190661478599221, '08'], [7.5647840531561465, '00'], [7.94299674267101, '18'], [12.380116959064328, '12'], [9.7119341563786, '04'], [6.782051282051282, '06'], [8.794258373205741, '05']]


[[28.676470588235293, '15'],
 [16.31756756756757, '13'],
 [12.380116959064328, '12'],
 [11.137546468401487, '02'],
 [10.684397163120567, '10'],
 [9.7119341563786, '04'],
 [9.692007797270955, '14'],
 [9.449744463373083, '17'],
 [9.190661478599221, '08'],
 [8.96474358974359, '11'],
 [8.804177545691905, '22'],
 [8.794258373205741, '05'],
 [8.749019607843136, '20'],
 [8.687258687258687, '21'],
 [7.948339483394834, '03'],
 [7.94299674267101, '18'],
 [7.713298791018998, '16'],
 [7.5647840531561465, '00'],
 [7.407801418439717, '01'],
 [7.163043478260869, '19'],
 [7.013274336283186, '07'],
 [6.782051282051282, '06'],
 [6.696793002915452, '23'],
 [6.653153153153153, '09']]

## Printing Values from a List of Lists
We'll display the top 5 highest number of comments per hour "Ask HN" post.

In [9]:
print("Top 5 Hours for Ask Posts Comments")
for row in sorted_swap[:5]:
    format_str = "{0}: {1:.2f} average comments per post."
    dt_hour = dt.datetime.strptime(row[1], "%H")
    str_hour = dt_hour.strftime("%H:%M")
    print(format_str.format(str_hour, row[0]))

Top 5 Hours for Ask Posts Comments
15:00: 28.68 average comments per post.
13:00: 16.32 average comments per post.
12:00: 12.38 average comments per post.
02:00: 11.14 average comments per post.
10:00: 10.68 average comments per post.


## Analysis of Average Comments per Hour

On average, the 15:00 (3:00pm EST, according to the dataset documentation time is stored in Eastern Standard Time) hour receives the most comments at 28.68 comments per post.  That's approximately a 76% increase in the number of comments between the highest and the second highest average comments by hour.

## Conclusion

We analyzed the "Ask HN" and "Show HN" posts to determine which type of post receives the most comments on average.  Based on our analysis, to maximize the amount of comments any post receives, we recommend that the post be categorized as "Ask HN" and created in the 15:00 to 16:00 (3:00pm - 4:00pm EST) time frame.

As a reminder, we only analyzed posts that received any comments.  Therefore, it's more accurate to state that *of the posts that received comments*, the "Ask HN" posts received the most comments on average between 3:00pm to 4:00pm EST.